In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### Extract Text from PDF Papers

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data/PDFs').load_data()
index = VectorStoreIndex.from_documents(documents)

### Generate Questions

In [ ]:
from llama_index.evaluation import DatasetGenerator

data_generator = DatasetGenerator.from_documents(documents)
questions = data_generator.generate_questions_from_nodes()
questions

In [ ]:
with open('data/llama_index_generated_questions.txt', 'w') as wf:
    for q in questions:
        wf.write("%s\n" % q)

### Evaluation

##### Are questions, answers, and context matching?

In [ ]:
import random

sampled_questions = random.sample(questions, 100)

In [ ]:
import pandas as pd
import time
from llama_index.evaluation import QueryResponseEvaluator
from llama_index.llm_predictor import LLMPredictor
from langchain.chat_models import ChatOpenAI
from llama_index import ServiceContext

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
evaluator = QueryResponseEvaluator(service_context=service_context)

query_engine = index.as_query_engine()
questions_and_answers = []
for q in sampled_questions:
    a = query_engine.query(q)
    eval_result = evaluator.evaluate(q, a)
    questions_and_answers.append({'Question': q, 'Answer': a.response, 'Match': eval_result})
    time.sleep(5)

qa_match_df = pd.DataFrame(questions_and_answers)
qa_match_df


In [ ]:
qa_match_df.to_json('questions_answers_english_validation.json', orient='records', indent=4)

In [ ]:
qa_match_df[qa_match_df['Match'] == 'YES']

### Evaluation Part 2: Questions in Vietnamese

In [ ]:
sampled_questions = random.sample(questions, 100)
sampled_questions = [q.strip() for q in sampled_questions]
sampled_questions

##### Translate sampled questions to Vietnamese

In [ ]:
def gpt4_translate_to_vi(question):
    """
    Translate a question to Vietnamese using GPT-4 model.

    :param question: Input question
    :return: Translated question
    """
    messages = [
        {
            "role": "system",
            "content": f"Translate the following question delimited by triple backticks to Vietnamese: {question}"
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0)
    return response["choices"][0]["message"]["content"].strip()

In [ ]:
# Using loop instead of list comprehension because token per minute rate exceeded
sampled_vi_questions = []
for q in sampled_questions:
    sampled_vi_questions.append(gpt4_translate_to_vi(q))
    time.sleep(5)
sampled_vi_questions

In [ ]:
vi_questions_and_answers = []
for q in sampled_vi_questions:
    a = query_engine.query(q)
    eval_result = evaluator.evaluate(q, a)
    vi_questions_and_answers.append({'Question': q, 'Answer': a.response, 'Match': eval_result})
    time.sleep(5)

vi_qa_match_df = pd.DataFrame(vi_questions_and_answers)
vi_qa_match_df

In [ ]:
import json

json_str = vi_qa_match_df.to_json(orient='records', force_ascii=False, date_format='iso')

formatted_json_str = json.dumps(json.loads(json_str), indent=4)

with open('questions_answers_vietnamese_validation.json', 'w', encoding='utf-8') as wf:
    wf.write(formatted_json_str)

In [ ]:
vi_qa_match_df[vi_qa_match_df['Match'] == 'YES']

### Save Preliminary Result

In [ ]:
index.set_index_id("vector_index")
index.storage_context.persist('preliminary-llama-index')

### Save List of PDF Papers

In [ ]:
import os

filenames = []
dir_path = 'data/PDFs/'
for f in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, f)):
        filenames.append(f)

with open('extracted_files.txt', 'w') as wf:
    for f in filenames:
        wf.write(f + '\n')

### Sample English Question

In [ ]:
english_question = "What water supply issues exist in Dania Beach?"
query_engine = index.as_query_engine()
response = query_engine.query(english_question)
print(response)

### Sample Vietnamese Question

In [ ]:
vietnamese_question = """Ở đâu trong vùng Mê Công là nơi có mức độ xâm nhập mặn cao nhất trong mùa khô 2019-2020
và gây thiệt hại nghiêm trọng đối với các mô hình trồng cây ăn trái và nuôi trồng thủy sản?"""
query_engine = index.as_query_engine()
response = query_engine.query(vietnamese_question)
print(response)

### Load Index from Local Filesystem

In [ ]:
from llama_index import StorageContext, load_index_from_storage

sc = StorageContext.from_defaults(persist_dir='preliminary-llama-index')
index2 = load_index_from_storage(sc, 'vector_index')

In [ ]:
# Test same sample Vietnamese question from above
vietnamese_question = """Ở đâu trong vùng Mê Công là nơi có mức độ xâm nhập mặn cao nhất trong mùa khô 2019-2020
và gây thiệt hại nghiêm trọng đối với các mô hình trồng cây ăn trái và nuôi trồng thủy sản?"""
query_engine = index2.as_query_engine()
response = query_engine.query(vietnamese_question)
print(response)